In [241]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import json
import os

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential, Model
from keras.layers import Conv1D, Dense, Dropout, Input, MaxPooling1D, Flatten
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras import utils
import tensorflow as tf
from tensorflow.keras import backend as K

utils.set_random_seed(42)

In [242]:
# Define Parameters
LOOKBACK = 4
HORIZON = 4
N_SPLITS = 2
BATCH_SIZE = 32
EPOCHS = 2
anomaly_weight = 0.001
model_name = 'cnn_univar1'

# Prepare data

In [243]:
with open('../config.json', 'r') as config_file:
    config = json.load(config_file)

df = pd.read_csv(os.path.join(config['data_path'], 'cell_undersampled_2.csv'))

print(df.shape)
print(df.columns)

(395914, 12)
Index(['timestamp', 'cell', 'bts', 'antenna', 'carrier', 'minRSSI',
       'pageSessions', 'ULvol', 'sessionSetupDur', 'sessionDur', 'blocks',
       'anomaly'],
      dtype='object')


In [244]:
# Sample 3 unique cells from the dataframe
sampled_cells = df['cell'].drop_duplicates().sample(n=3, random_state=42)
mini_df = df[df['cell'].isin(sampled_cells)]

print(mini_df.shape)
print()
print(mini_df.groupby('cell')['anomaly'].mean().round(3))

(8197, 12)

cell
238_0_1    0.121
341_2_0    0.129
599_1_0    0.111
Name: anomaly, dtype: float64


In [245]:
temporal_X = []
static_X = []

# Funcs

In [246]:
# Time series split function (Expanding Window)
def time_series_split(df, n_splits=N_SPLITS, test_size=0.2):
    df = df.sort_values('timestamp')
    test_split_index = int(len(df) * (1 - test_size))
    train_val_df = df.iloc[:test_split_index]
    test_df = df.iloc[test_split_index:]

    tscv = TimeSeriesSplit(n_splits=n_splits)
    splits = [(train_val_df.iloc[train_index], train_val_df.iloc[val_index]) for train_index, val_index in tscv.split(train_val_df)]
    return splits, test_df

In [247]:
def create_sequences(df, lookback=LOOKBACK, horizon=HORIZON, static_X=static_X, temporal_X=temporal_X):
    X, y, cell_id = [], [], []

    for cell in df['cell'].unique():
        cell_df = df[df['cell'] == cell]

        for i in range(lookback, len(cell_df) - horizon + 1):
            # Time-variant features
            X_seq = cell_df.iloc[i - lookback:i][['minRSSI'] + temporal_X].values

            # Static features (replicated across lookback steps)
            static_seq = (
                np.tile(cell_df.iloc[i][static_X].values, (lookback, 1))
                if static_X else []
            )

            # Combine features
            X_combined = np.concatenate([X_seq, static_seq], axis=1) if static_X else X_seq

            # Target and anomaly labels
            y_seq = cell_df.iloc[i:i + horizon]['minRSSI'].values
            anomaly_seq = cell_df.iloc[i:i + horizon]['anomaly'].values

            # Append anomaly indicator to the target
            y_seq_with_anomaly = np.column_stack((y_seq, anomaly_seq))

            X.append(X_combined)
            y.append(y_seq_with_anomaly)
            cell_id.append(cell_df.iloc[i:i + horizon]['cell'].values)

    print(f"Sequences created: X shape = {np.array(X).shape}, y shape = {np.array(y).shape}", end='\n\n')
    return np.array(X), np.array(y), np.array(cell_id)

In [248]:
def scale_data_split(train_df, val_df, temporal_features=temporal_X, static_features=static_X):
    scaler_temporal = StandardScaler()
    scaler_static = MinMaxScaler()
    scaler_target = StandardScaler()

    # Scale time-variant features
    if temporal_features:
        train_df[temporal_features] = scaler_temporal.fit_transform(train_df[temporal_features])
        val_df[temporal_features] = scaler_temporal.transform(val_df[temporal_features])

    # Scale time-invariant features
    if static_features:
        train_df[static_features] = scaler_static.fit_transform(train_df[static_features])
        val_df[static_features] = scaler_static.transform(val_df[static_features])

    # Scale minRSSI separately (target variable)
    train_df['minRSSI'] = scaler_target.fit_transform(train_df[['minRSSI']])
    val_df['minRSSI'] = scaler_target.transform(val_df[['minRSSI']])

    return train_df, val_df, scaler_target, scaler_temporal, scaler_static

In [249]:
def weighted_mae(y_true, y_pred, weights):
    # Extract the minRSSI values (first column in y_true)
    y_true_values = y_true[:, :, 0]  # minRSSI values
    anomaly_mask = y_true[:, :, 1]   # Anomaly indicators (0 or 1)

    # Compute the absolute error
    mae = tf.abs(y_true_values - y_pred)

    # Apply the weighting for anomalies
    weight_matrix = 1 + anomaly_mask * (weights - 1)  # Regular instances: weight=1, Anomalies: weight=weights
    weighted_mae = mae * weight_matrix

    # Return the mean weighted MAE
    return tf.reduce_mean(weighted_mae)

In [250]:
def build_cnn(lookback, horizon, n_features, anomaly_weight=1.0):
    print("build_cnn activated")
    model = Sequential()
    model.add(Input(shape=(lookback, n_features)))

    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(horizon))  # Outputs horizon predictions

    # Compile with the custom weighted loss function
    model.compile(
        optimizer='adam',
        loss=lambda y_true, y_pred: weighted_mae(y_true, y_pred, weights=anomaly_weight),
    )
    return model


In [251]:
def train_validate(splits, lookback, horizon, anomaly_weight=1.0):
    results = []
    scalers = {}
    total_training_time = 0
    threshold = 0.1  # Example threshold for anomaly detection

    for i, (train_df, val_df) in enumerate(splits):
        print(f"Processing Split {i + 1}/{len(splits)}")

        # Scale data
        scaled_train, scaled_val, scaler_target, scaler_temporal, scaler_static = scale_data_split(
            train_df.copy(), val_df.copy()
        )
        print(f"Scaled train shape: {scaled_train.shape}, Scaled val shape: {scaled_val.shape}")

        scalers = {
            'scaler_target': scaler_target,
            'scaler_temporal': scaler_temporal,
            'scaler_static': scaler_static
        }

        # Create sequences
        X_train, y_train, _ = create_sequences(scaled_train, LOOKBACK, HORIZON)
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        X_val, y_val, _ = create_sequences(scaled_val, LOOKBACK, HORIZON)

        # Ensure data type compatibility
        X_train, y_train = X_train.astype(np.float32), y_train.astype(np.float32)
        X_val, y_val = X_val.astype(np.float32), y_val.astype(np.float32)

        n_features = X_train.shape[2]

        # Build the model
        model = build_cnn(lookback, horizon, n_features, anomaly_weight)

        # Early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Start timer
        start_time = time.time()

        # Train the model
        history = model.fit(
            X_train, y_train,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=1
        )

        # End timer
        split_training_time = time.time() - start_time
        total_training_time += split_training_time

        # Evaluate
        y_pred = model.predict(X_val)
        y_val_actual = y_val[:, :, 0]  # Extract actual values from y_val (excluding anomaly indicator)
        y_pred_actual = y_pred
        y_val_anomaly = y_val[:, :, 1]  # Extract anomaly indicator

        # Overall MAE and RMSE
        mae = mean_absolute_error(y_val_actual, y_pred_actual)
        rmse = np.sqrt(mean_squared_error(y_val_actual, y_pred_actual))

        # Anomalous MAE and RMSE
        anomalous_indices = y_val_anomaly == 1
        mae_anom = mean_absolute_error(y_val_actual[anomalous_indices], y_pred_actual[anomalous_indices])
        rmse_anom = np.sqrt(mean_squared_error(y_val_actual[anomalous_indices], y_pred_actual[anomalous_indices]))

        # Binary anomaly predictions
        anomaly_predicted = np.abs(y_val_actual - y_pred_actual) > threshold

        # Confusion Matrix
        tp = np.sum((anomaly_predicted & (y_val_anomaly == 1)))
        fp = np.sum((anomaly_predicted & (y_val_anomaly == 0)))
        tn = np.sum((~anomaly_predicted & (y_val_anomaly == 0)))
        fn = np.sum((~anomaly_predicted & (y_val_anomaly == 1)))

        # Recall
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        results.append({
            'split': i + 1,
            'MAE': mae,
            'RMSE': rmse,
            'MAE_anom': mae_anom,
            'RMSE_anom': rmse_anom,
            'TP': tp,
            'FP': fp,
            'TN': tn,
            'FN': fn,
            'Recall': recall
        })

    # Summarize
    avg_mae = np.mean([res['MAE'] for res in results])
    avg_rmse = np.mean([res['RMSE'] for res in results])
    avg_mae_anom = np.mean([res['MAE_anom'] for res in results])
    avg_rmse_anom = np.mean([res['RMSE_anom'] for res in results])
    avg_recall = np.mean([res['Recall'] for res in results])

    summary_results = {
        'Average MAE': avg_mae,
        'Average RMSE': avg_rmse,
        'Average MAE_anom': avg_mae_anom,
        'Average RMSE_anom': avg_rmse_anom,
        'Average Recall': avg_recall,
        'Total Training Time': f"{total_training_time // 60}m {total_training_time % 60:.2f}s"
    }

    return summary_results, model, scalers

# Run the training

In [252]:
splits, test_set = time_series_split(mini_df, N_SPLITS)

for i, (train, val) in enumerate(splits):
    print(f"Split {i + 1}:")
    print(f"  Train set shape: {train.shape}")
    print(f"  Validation set shape: {val.shape}")

print(f"Test set shape: {test_set.shape}")

Split 1:
  Train set shape: (2187, 12)
  Validation set shape: (2185, 12)
Split 2:
  Train set shape: (4372, 12)
  Validation set shape: (2185, 12)
Test set shape: (1640, 12)


In [253]:
# Train and evaluate the model across all splits
summary_results, model, scalers = train_validate(splits, LOOKBACK, HORIZON)

Processing Split 1/2
Scaled train shape: (2187, 12), Scaled val shape: (2185, 12)
Sequences created: X shape = (2166, 4, 1), y shape = (2166, 4, 2)

X_train shape: (2166, 4, 1), y_train shape: (2166, 4, 2)
Sequences created: X shape = (2164, 4, 1), y shape = (2164, 4, 2)

build_cnn activated
Epoch 1/2
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6577 - val_loss: 0.1449
Epoch 2/2
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3236 - val_loss: 0.1356
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step
Processing Split 2/2
Scaled train shape: (4372, 12), Scaled val shape: (2185, 12)
Sequences created: X shape = (4351, 4, 1), y shape = (4351, 4, 2)

X_train shape: (4351, 4, 1), y_train shape: (4351, 4, 2)
Sequences created: X shape = (2164, 4, 1), y shape = (2164, 4, 2)

build_cnn activated
Epoch 1/2
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4830 - val_loss: 0.2956
Epoch 2/2
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - loss: 0.2491 - val_loss: 0.2818
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 863us

In [255]:
print("\nCross-Validation Results:")
for metric, value in summary_results.items():
    try:
        print(f"{metric}: {float(value):.4f}")
    except ValueError:
        print(f"{metric}: {value}")


Cross-Validation Results:
Average MAE: 0.2065
Average RMSE: 0.3924
Average MAE (Anomalies): 0.8375
Average RMSE (Anomalies): 1.0247
Average Recall: 0.9237
Total Training Time: 0.0m 2.13s
